In [8]:
# PAD-US → % of tract covered by open-access parks (CONUS only)
# Outputs saved into OUT_DIR with filenames defined inline below.

from pathlib import Path
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

#--------------------------------------------------------------------
# Required paths 
#--------------------------------------------------------------------
GDB        = Path(r"C:\CVI\source\PADUS4_1VectorAnalysis_PADUS_Only\PADUS4_1VectorAnalysis_PADUS_Only.gdb")
TRACTS_SHP = Path(r"C:\CVI\resources\cb_2020_us_tract_500k\cb_2020_us_tract_500k.shp")
OUT_DIR    = Path(r"C:\CVI\harmonized\PARKS")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Equal-area CRS for accurate area math (CONUS Albers)
EA_CRS = 5070

#--------------------------------------------------------------------
# 1) Load PAD-US analysis layer
#--------------------------------------------------------------------
import fiona
layers = fiona.listlayers(GDB.as_posix())
layer  = next((L for L in layers if "Simp_SingP" in L), layers[0])

try:
    import pyogrio  # faster if available
    parks_all = gpd.read_file(GDB.as_posix(), layer=layer, engine="pyogrio")
except Exception:
    parks_all = gpd.read_file(GDB.as_posix(), layer=layer)

#--------------------------------------------------------------------
# 2) Filter to “public-use” polygons
#    Pub_Access: OA=open; include Fee/Designation/Easement only
#--------------------------------------------------------------------
parks = parks_all[
    parks_all["Pub_Access"].isin(["OA"]) &
    parks_all["FeatClass"].isin(["Fee", "Designation", "Easement"])
].copy()

# Optional small preview (sampled)
if len(parks) > 0:
    ax = parks.sample(min(len(parks), 15000), random_state=42)\
              .plot(edgecolor="black", linewidth=0.1, figsize=(10, 7))
    ax.set_title("PAD-US Open-Access Parks (sampled)")
    ax.set_axis_off()
    plt.tight_layout()
    plt.savefig(OUT_DIR / "PARKS_preview.png", dpi=150)
    plt.close()

#--------------------------------------------------------------------
# 3) Load 2020 tracts and keep CONUS (drop AK, HI, PR)
#--------------------------------------------------------------------
tracts = gpd.read_file(TRACTS_SHP.as_posix())[
    ["GEOID", "STATEFP", "geometry"]
]
tracts = tracts[~tracts["STATEFP"].isin(["02", "15", "72"])].copy()

#--------------------------------------------------------------------
# 4) Reproject to equal-area CRS (EPSG:5070) for area math
#--------------------------------------------------------------------
parks_ea  = parks.to_crs(EA_CRS)
tracts_ea = tracts.to_crs(EA_CRS)

#--------------------------------------------------------------------
# 5) Compute % coverage by tract
#   a) tract areas
#   b) prefilter to tract–park candidates (sjoin)
#   c) exact intersection (overlay)
#   d) sum intersection area by GEOID
#   e) percent = (sum_intersection / tract_area) * 100
#--------------------------------------------------------------------
# (a) tract areas
tracts_ea["tract_area_m2"] = tracts_ea.area

# (b) candidate pairs
hits = gpd.sjoin(
    tracts_ea[["GEOID", "geometry"]],
    parks_ea[["geometry"]],
    how="inner", predicate="intersects"
)
tracts_sub = tracts_ea[tracts_ea["GEOID"].isin(hits["GEOID"].unique())].copy()

# (c) exact intersection
inter = gpd.overlay(
    parks_ea[["geometry"]],
    tracts_sub[["GEOID", "geometry"]],
    how="intersection",
    keep_geom_type=True
)

# (d) aggregate intersection area by tract
inter["part_area_m2"] = inter.area
by_tract = inter.groupby("GEOID", as_index=False)["part_area_m2"].sum()

# (e) percent coverage and CSV
out = tracts_ea[["GEOID", "tract_area_m2"]].merge(by_tract, on="GEOID", how="left")
out["part_area_m2"] = out["part_area_m2"].fillna(0.0)
out["park"] = (out["part_area_m2"] / out["tract_area_m2"]).clip(0, 1) * 100.0

out[["GEOID", "park"]].rename(columns={"GEOID": "geoid"}) \
    .to_csv(OUT_DIR / "PARKS_PADUS_open_conus_tract_2020.csv", index=False)

#--------------------------------------------------------------------
# 6) Summary text + QA plots
#--------------------------------------------------------------------
summary_text = out["park"].describe(percentiles=[0.25, 0.5, 0.75, 0.9, 0.99]).to_string()
with open(OUT_DIR / "PARKS_summary.txt", "w", encoding="utf-8") as f:
    f.write("PARKS (PAD-US open-access) – Tract % coverage (CONUS)\n\n")
    f.write(summary_text + "\n")

# Histogram
plt.figure(figsize=(9, 5))
plt.hist(out["park"], bins=60)
plt.title("Distribution of % Tract Covered by Open-Access Parks")
plt.xlabel("% coverage"); plt.ylabel("Count")
plt.tight_layout()
plt.savefig(OUT_DIR / "PARKS_hist.png", dpi=150)
plt.close()

# Choropleth (continuous colorbar; 1st–99th percentile stretch)
tracts_plot = tracts_ea.merge(out[["GEOID", "park"]], on="GEOID")
vmin = float(np.nanpercentile(tracts_plot["park"], 1))
vmax = float(np.nanpercentile(tracts_plot["park"], 99))
ax = tracts_plot.plot(column="park", vmin=vmin, vmax=vmax, legend=True,
                      linewidth=0, figsize=(10, 7))
ax.set_title("Open-Access Parks – % of Tract Covered (CONUS, PAD-US)")
ax.set_axis_off()
plt.tight_layout()
plt.savefig(OUT_DIR / "PARKS_quickmap.png", dpi=150)
plt.close()

print("Done. Outputs in:", OUT_DIR)


Done. Outputs in: C:\CVI\harmonized\PARKS
